In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
import requests
from langchain_core.runnables import RunnablePassthrough

In [2]:
# OLLAMA_URL = "http://10.246.167.250:9200"
OLLAMA_URL = "http://127.0.0.1:11434"

payload = requests.get(f"{OLLAMA_URL}/api/tags").json()
model_names = [model['name'] for model in payload['models']]
model_names = ("\n".join(model_names)).rstrip("\n")
print(model_names)

bakllava:7b
llama2:13b
llama2:7b
mistral:7b


In [3]:
llm = Ollama(
    model="llama2:7b",
    base_url=OLLAMA_URL,
    temperature=0.5,
)

In [4]:
template = """Use the following context strictly to formulate your answers:
Context: {context}
Question: {question}

Make sure to answer this briefly. Answer strictly in {language} only and mention the reference document name.
Answer:
"""

prompt = PromptTemplate.from_template(template)

output_parser = StrOutputParser()

In [5]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

loader = DirectoryLoader('../documents', glob="**/*.txt", show_progress=True, loader_cls=TextLoader)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
embedding = OllamaEmbeddings(
    model="llama2:7b",
    base_url=OLLAMA_URL,
)
vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding)

100%|██████████| 2/2 [00:00<00:00, 1001.98it/s]


In [6]:
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(f"Document: {doc.metadata['source']}\n{doc.page_content}" for doc in docs)

In [7]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough(), "language": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

output = chain.invoke({
    "question": "Who is my best friend?",
    "language": "English"
})
print(output)

My best friend is Shiva, as stated in the "other.txt" document.


In [ ]:
rag = retriever | format_docs
print(rag.invoke("OK"))